In [27]:
import os
import numpy as np
import math

data_dir = '/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/test/'
all_files = os.listdir(data_dir)
all_files = [os.path.join(data_dir, _path) for _path in all_files]
print(all_files)
pred_len = 12
obs_len = 8
threshold = 0.002
min_ped = 1

['/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/test/biwi_eth.txt']


In [28]:
def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

In [29]:
def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len) #linspace(start, stop, number)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1] #returns residual for a polynomial fit for x pos
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1] #same for y pos 
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0


In [102]:
import pdb



num_peds_in_seq = []
seq_list = []
seq_list_rel = []
loss_mask_list = []
non_linear_ped = []
for path in all_files:
    print(path)
    data = read_file(path)
    frames = np.unique(data[:, 0]).tolist()
    frame_data = []
    for frame in frames:
        frame_data.append(data[frame == data[:, 0], :])
    print(len(frames), ' frames')
    skip = 1
    seq_len = 20
    num_sequences = int(math.ceil((len(frames) - seq_len + 1) / skip))
    for idx in range(0, num_sequences * skip + 1, skip):
        curr_seq_data = np.concatenate(frame_data[idx:idx + seq_len], axis=0)
        peds_in_curr_seq = np.unique(curr_seq_data[:, 1]) #pedestrian ids per sequence (of frames)
        #print(len(peds_in_curr_seq))
        curr_seq_rel = np.zeros((len(peds_in_curr_seq), 2, seq_len))
        curr_seq = np.zeros((len(peds_in_curr_seq), 2, seq_len))
        curr_loss_mask = np.zeros((len(peds_in_curr_seq),seq_len))
        num_peds_considered = 0
        _non_linear_ped = []
        for _, ped_id in enumerate(peds_in_curr_seq):
            curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] ==
                                         ped_id, :]
            curr_ped_seq = np.around(curr_ped_seq, decimals=4)
            pad_front = frames.index(curr_ped_seq[0, 0]) - idx
            pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
            if pad_end - pad_front != seq_len:
                continue
            curr_ped_seq = np.transpose(curr_ped_seq[:, 2:])
            curr_ped_seq = curr_ped_seq #shape = (2, seq_len) matrix: xpos row, ypos row for each pedestrian id
            # Make coordinates relative
            rel_curr_ped_seq = np.zeros(curr_ped_seq.shape)
            rel_curr_ped_seq[:, 1:] = \
                curr_ped_seq[:, 1:] - curr_ped_seq[:, :-1] #normalized per sequence
            print(curr_ped_seq)
            #print(rel_curr_ped_seq)
            _idx = num_peds_considered
            curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq
            curr_seq_rel[_idx, :, pad_front:pad_end] = rel_curr_ped_seq
            # Linear vs Non-Linear Trajectory
            _non_linear_ped.append(
                poly_fit(curr_ped_seq, pred_len, threshold))
            curr_loss_mask[_idx, pad_front:pad_end] = 1
            num_peds_considered += 1


        if num_peds_considered > min_ped:
            non_linear_ped += _non_linear_ped
            num_peds_in_seq.append(num_peds_considered)
            loss_mask_list.append(curr_loss_mask[:num_peds_considered])
            seq_list.append(curr_seq[:num_peds_considered])
            seq_list_rel.append(curr_seq_rel[:num_peds_considered])

            
print('done') 
print(np.sum(num_peds_in_seq))
#print(num_peds_in_seq[:10])
num_seq = len(seq_list)
seq_list = np.concatenate(seq_list, axis=0)
seq_list_rel = np.concatenate(seq_list_rel, axis=0)
loss_mask_list = np.concatenate(loss_mask_list, axis=0)
non_linear_ped = np.asarray(non_linear_ped)
print('non_lin: ', non_linear_ped.shape)
print(non_linear_ped[:10])
print(seq_list.shape)
print(loss_mask_list.shape)
obs_traj = seq_list[:,:,:obs_len]
_len = [len(seq) for seq in obs_traj]
cum_start_idx = [0] + np.cumsum(_len).tolist()
seq_start_end = [[start, end] for start, end in zip(cum_start_idx, cum_start_idx[1:])]
#print(seq_list[1,:,:])
print(sum(loss_mask_list))
cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
start_end = [(start, end) for start, end in zip(cum_start_idx, cum_start_idx[1:])]
print(start_end)
print(seq_start_end)

/Users/chandrayeebasu/Documents/insight/project/sgan/datasets/eth/test/biwi_eth.txt
876  frames
[[13.64 12.09 11.37 10.31  9.57  8.73  7.94  7.17  6.47  5.86  5.24  4.87
   4.51  4.2   3.95  3.47  2.82  2.01  1.28  0.54]
 [ 5.8   5.75  5.8   5.97  6.24  6.34  6.5   6.62  6.68  6.82  6.98  7.16
   7.58  7.3   7.71  7.86  8.    8.    7.82  7.4 ]]
[[12.09 11.37 10.31  9.57  8.73  7.94  7.17  6.47  5.86  5.24  4.87  4.51
   4.2   3.95  3.47  2.82  2.01  1.28  0.54 -0.18]
 [ 5.75  5.8   5.97  6.24  6.34  6.5   6.62  6.68  6.82  6.98  7.16  7.58
   7.3   7.71  7.86  8.    8.    7.82  7.4   7.06]]
[[11.37 10.31  9.57  8.73  7.94  7.17  6.47  5.86  5.24  4.87  4.51  4.2
   3.95  3.47  2.82  2.01  1.28  0.54 -0.18 -0.83]
 [ 5.8   5.97  6.24  6.34  6.5   6.62  6.68  6.82  6.98  7.16  7.58  7.3
   7.71  7.86  8.    8.    7.82  7.4   7.06  6.43]]
[[10.31  9.57  8.73  7.94  7.17  6.47  5.86  5.24  4.87  4.51  4.2   3.95
   3.47  2.82  2.01  1.28  0.54 -0.18 -0.83 -1.52]
 [ 5.97  6.24  6.34  6.5   6

In [31]:
traj_len = 5
traj = np.random.randn(5)
t = np.linspace(0, traj_len - 1, 5)
res_x = np.polyfit(t, traj, 2, full=True)[1]
print(res_x)

[1.40534153]


In [101]:
import json

fdir = ['/Users/chandrayeebasu/Documents/insight/carladata/data/ma_datum_00000000.json', '/Users/chandrayeebasu/Documents/insight/carladata/data/ma_datum_00000007.json', '/Users/chandrayeebasu/Documents/insight/carladata/data/ma_datum_00000005.json']

agent_considered = 0
seq_list = []
start_end = []
for f in fdir:
    with open(f) as fi:
        d = json.load(fi)
    agent_pasts = np.asarray(d['agent_pasts'])
    agent_futures = np.asarray(d['agent_futures'])
    player_past = np.asarray(d['player_past'])
    player_future = np.asarray(d['player_future'])
    total_seq = np.concatenate((agent_pasts[:,:,:2], agent_futures[:,:,:2]), axis=1)
    start = agent_considered
    num_agent = agent_pasts.shape[0]
    agent_considered += num_agent
    start_end.append((start,agent_considered))
    total_seq_re = []
    for i in range(total_seq.shape[0]):
        total_seq_re.append(total_seq[i,:,:].T)
    player_seq = np.concatenate((player_past[:,:2], player_future[:,:2]), axis=0)
    total_seq_re = np.asarray(total_seq_re)
    seq_list.append(total_seq_re)
seq_list = np.vstack(seq_list)
seq_list_rel = np.zeros(seq_list.shape)
non_linear_ped = np.zeros(seq_list.shape[0])
for i in range(seq_list.shape[0]):
    seq_list_rel[i,:,1:] = seq_list[i,:,1:] - seq_list[i,:,:-1]
    non_linear_ped[i] = poly_fit(seq_list[i],20,threshold)
loss_mask_list = np.ones((seq_list.shape[0],seq_list.shape[2]))
obs_len = 10
obs_seq_list = seq_list[:,:,:obs_len]
#print(obs_seq_list.shape)
#print([seq for seq in obs_seq_list])
_len = [len(seq) for seq in obs_seq_list]
cum_start_idx = [0] + np.cumsum(_len).tolist()
seq_start_end = [[start, end] for start, end in zip(cum_start_idx, cum_start_idx[1:])]
print(seq_start_end)
print(start_end)

[[0, 2], [2, 4], [4, 6], [6, 8], [8, 10], [10, 12], [12, 14], [14, 16], [16, 18], [18, 20], [20, 22], [22, 24]]
[(0, 4), (4, 8), (8, 12)]
